# Домашнее задание «Функции потерь и оптимизация» 

## Задание

### Цель: 
изучить применение методов оптимизации для решения задачи классификации
### Описание задания:
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.


In [63]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import time

## Этапы работы:

### 1. Загрузите данные 
Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [64]:
from sklearn.datasets import load_iris
iris = load_iris()

In [65]:
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
# удаление строк класса 0 
df = df[df.target != 0]
df.head(-5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
140,6.7,3.1,5.6,2.4,2
141,6.9,3.1,5.1,2.3,2
142,5.8,2.7,5.1,1.9,2
143,6.8,3.2,5.9,2.3,2


In [66]:
df['target'] = df['target'].map({1: 0, 2: 1})
# df

In [67]:
df['class'] = df['target'].map({0: 'Iris Versicolor', 1: 'Iris Virginica'})
# df

In [68]:
X = df.drop(['target', 'class'], axis=1)
# X['theta0'] = 0
y = df['target']
X.shape, y

((100, 4),
 50     0
 51     0
 52     0
 53     0
 54     0
       ..
 145    1
 146    1
 147    1
 148    1
 149    1
 Name: target, Length: 100, dtype: int64)

In [69]:
# нормирование входных данных
def standardize(X):
    std_X = (X - X.mean(0) ) / X.std(0)
    print(f'Среднее отклонение:\n{std_X.mean(axis=0)}\n')
    print(f'Стандартное отклонение:\n{std_X.std(axis=0)}')
    return std_X

In [70]:
X = standardize(X)

Среднее отклонение:
sepal length (cm)    3.356204e-15
sepal width (cm)    -3.599343e-15
petal length (cm)   -7.338574e-16
petal width (cm)    -1.260103e-15
dtype: float64

Стандартное отклонение:
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


### 2. Реализуйте логистическую регрессию ( без использования метода LogisticRegression из библиотеки) 
*Оформите в виде класса с методами.*

In [71]:
class LogisticRegression:
    def __init__(self, learning_rate=0.001, epochs=1000, method='gd', epsilon=1e-10):
        # инициализация скорости обучения, количества итераций, метода оптимизации, сглаживающего параметра
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.method = method
        self.epsilon = epsilon
        
    def add_intercept(self, X): # добавление коэффициента для свободного веса
        return np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)


    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))


    def logloss(self, y, y_pred): # функция потерь 
        lloss0 = np.sum(np.log(1 - self.y_pred[self.y == 0]) + self.epsilon)
        lloss1 = np.sum(np.log(self.y_pred[self.y == 1]) + self.epsilon)
        return -(lloss0 + lloss1)/len(self.y)
       
    
    def fit(self, X, y):

        np.random.seed(42)
        self.weights = np.random.randn(X.shape[1])
#         self.weights = np.zeros(X.shape[1])     # начальные веса могут быть нулями

        for i in range(self.epochs):
        
            y_pred = self.sigmoid(np.dot(X, self.weights))
            gradient = np.dot(X.T, (y_pred - y)) / y.size

            if self.method == 'gd': # оптимизация по методу градиентного спуска
                self.weights -= self.learning_rate * gradient

            if self.method == 'rmsprop': # оптимизация по методу скользящего среднего
                beta = 0.9 # коэффициент сохранения
                v = beta * np.zeros(self.weights.shape) + (1 - beta) * gradient**2
                self.weights -= self.learning_rate * gradient / (np.sqrt(v) + self.epsilon)

        
    def predict(self, X, sep = 0.5): # передсказание класса
        prob = self.sigmoid(np.dot(X, self.weights))
        return [1 if x >= sep else 0 for x in prob] 
#         return np.round(self.sigmoid(X, self.theta), 3) # вывод вероятности для каждого элемента

    def accuracy(self, X, y):
        return round((self.predict(X) == y).mean(), 2)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### 2.1 Реализуйте метод градиентного спуска. 
Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.

In [73]:
lr_gd = LogisticRegression() 
# время обучения модели
%time lr_gd.fit(X_train, y_train)

Wall time: 275 ms


In [74]:
# точность
lr_gd.accuracy(X_test, y_test)

0.95

#### 2.2 Реализуйте метод скользящего среднего 
(Root Mean Square Propagation, RMSProp).

In [75]:
lr_rmsp = LogisticRegression(method='rmsprop') 
# время обучения модели
%time lr_rmsp.fit(X_train, y_train)

Wall time: 278 ms


In [76]:
# точность
lr_rmsp.accuracy(X_test, y_test)

0.8

##### Метод ускоренного градиента Нестерова реализовать не удалось

### 3. Сравните значение метрик для реализованных методов оптимизации.  

| Метод | Точность | Время работы (ms) |
| :------- |:-------------:| :--------:|
| Градиентный спуск |   0.95  | 275 |
| RMSProp |  0.8 | 278 |


## Вывод

Результат: получены навыки реализации методов оптимизации в задаче бинарной классификации. Пройденные методы оптимизации используются и в нейросетях.

На конкретном премере методы оптимизации очень чувствительны к значениям тренировочной выборке за счет ее малого размера. 
Время работы алгоритмов сильно колеблится, а вот точность обычного Градиентного спуска значительно выше.